In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import yfinance as yf
import matplotlib.pyplot as plt
import datetime
import math

"""
Saving all the plots on Google Colab.
"""
from google.colab import drive
drive.mount('/content/drive')


def get_details(ticker):
    # Combine all the functions into a single function.
    def calculate_difference(data, d):
        """
        Input:
        data: A dataframe containing all the data related to stocks.
        d: A parameter, to adjust how many days to consider in the past.
        """
        difference = [] # Store the deltas (changes) for the last d days.
        for index in range(d):
            difference.append(data["Close"].shift(index) - data["Open"].shift(index))
        difference = difference[::-1]
        return np.vstack(difference).T

    def prepare_data(data, d):
        """
        A combined function to create both the feature vector and the corresponding labels.
        This shall return both the features and labels together, using the past prices.
        """
        difference = calculate_difference(data, d)
        # Create the feature vector.
        features = difference
        # Create the vector of labels.
        labels = (data["Close"] - data["Open"]).values.reshape(-1, 1)
        # Adjust labels to calibrate the dataset properly.
        labels = np.roll(labels, -1)

        # Return the features along with their respective labels, and the feature vector of the input.
        return features[d:-1], labels[d:-1], features[-1]

    def fetch_data(ticker_symbol, start_date, end_date):
        """
        Fetch the historical records using yfinance.
        """
        end_date = (datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)
        return stock_data

    def predict_next_day(to_predict):
        # Scale and reshape the feature vector which needs to be predicted.
        to_predict = to_predict.reshape((1, to_predict.shape[0]))
        to_predict = scaler.fit_transform(to_predict)
        to_predict = to_predict.reshape((1, 1, to_predict.shape[1]))

        # Use the trained LSTM model to predict the output for the feature vector.
        predicted_output = model.predict(to_predict)
        print("Predicted output:", predicted_output)

    # Set the parameters.
    d = 7                        # Price difference needs to be caluclated over the past 30 days.
    ticker_symbol = ticker       # Ticker symbol of the stock.
    start_date = "2020-01-01"    # Start date for fetching data.
    end_date = (datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # End date for fetching data.

    # Fetch historical data for the stock, and get its features v/s labels.
    data = fetch_data(ticker_symbol, start_date, end_date)
    features, labels, to_predict = prepare_data(data, d)

    # Split data into training and testing sets.
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Scale all the feature vectors.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)

    # Build LSTM Model.
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(100, input_shape=(1, X_train_scaled.shape[1])),
        tf.keras.layers.LSTM(70),
        tf.keras.layers.LSTM(40),
        tf.keras.layers.LSTM(10),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(5),
        tf.keras.layers.Dense(1)
    ])

    # Compile the model.
    model.compile(optimizer="adam", loss="mean_squared_error")

    # Train the model.
    history = model.fit(X_train_scaled, y_train, epochs=100, verbose=1)
    # Get the predictions.
    y_pred = model.predict(X_test_scaled)

    # Visualising the results.
    plt.scatter(y_pred, y_test)
    plt.xlabel("Predicted value")
    plt.ylabel("Actual value")

    # Plot y = x line on the same plot to visualise the accuracy.
    plt.plot(y_pred, y_pred, color='red')

    plt.title('Predicted vs Actual Deltas')
    plt.legend()
    plt.grid(True)
    return plt

nifty50_tickers = [
    "ADANIPORTS.NS", "ASIANPAINT.NS", "AXISBANK.NS", "BAJAJ-AUTO.NS", "BAJFINANCE.NS",
    "BAJAJFINSV.NS", "BPCL.NS", "BHARTIARTL.NS", "BRITANNIA.NS", "CIPLA.NS",
    "COALINDIA.NS", "DIVISLAB.NS", "DRREDDY.NS", "EICHERMOT.NS", "GRASIM.NS",
    "HCLTECH.NS", "HDFC.NS", "HDFCBANK.NS", "HDFCLIFE.NS", "HEROMOTOCO.NS",
    "HINDALCO.NS", "HINDUNILVR.NS", "ICICIBANK.NS", "ITC.NS", "IOC.NS",
    "INDUSINDBK.NS", "INFY.NS", "JSWSTEEL.NS", "KOTAKBANK.NS", "LT.NS",
    "M&M.NS", "MARUTI.NS", "NTPC.NS", "NESTLEIND.NS", "ONGC.NS",
    "POWERGRID.NS", "RELIANCE.NS", "SBILIFE.NS", "SHREECEM.NS", "SBIN.NS",
    "SUNPHARMA.NS", "TCS.NS", "TATAMOTORS.NS", "TATASTEEL.NS", "TECHM.NS",
    "TITAN.NS", "UPL.NS", "ULTRACEMCO.NS", "WIPRO.NS", "ZEEL.NS"
]

for ticker in nifty50_tickers:
    plt.figure()
    get_details(ticker)
    plt.savefig(f'/content/drive/My Drive/trading/self_attempt/plots/{ticker}.png')

[*********************100%%**********************]  1 of 1 completed

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ValueError: Input 0 of layer "lstm_83" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 100)

<Figure size 640x480 with 0 Axes>